In [1]:
import keras
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import cv2
import glob
%matplotlib inline

Using TensorFlow backend.


In [2]:
from keras.preprocessing.image import ImageDataGenerator

#Image Data Generator w/ no augmentation
#Scaling for pixels
piece_train_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_test_datagen = ImageDataGenerator(
    rescale = 1./255)
piece_valid_datagen = ImageDataGenerator(
    rescale = 1./255)


#Flow data from directory

piece_train_iter = piece_train_datagen.flow_from_directory(
    directory = 'data/piece_data/train',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

piece_test_iter = piece_test_datagen.flow_from_directory(
    directory = 'data/piece_data/test',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    shuffle=False,
    seed=42
)

piece_valid_iter = piece_valid_datagen.flow_from_directory(
    directory = 'data/piece_data/valid',
    target_size = (135,135),
    color_mode = 'grayscale',
    class_mode = 'categorical',
    seed=42
)

Found 1227 images belonging to 7 classes.
Found 335 images belonging to 7 classes.
Found 331 images belonging to 7 classes.


In [3]:
#Define NN architecture

from keras.layers import Conv2D, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from keras.layers import Dropout, Flatten, Dense
from keras.models import Sequential

piece_model = Sequential()
piece_model.add(Conv2D(filters=16, kernel_size=2, padding='same', activation='relu', 
                        input_shape=(135, 135, 1)))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=32, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(BatchNormalization())
piece_model.add(Conv2D(filters=64, kernel_size=2, padding='same', activation='relu'))
piece_model.add(MaxPooling2D(pool_size=2))
piece_model.add(GlobalAveragePooling2D())
piece_model.add(Dense(7, activation='softmax'))


piece_model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 135, 135, 16)      80        
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 67, 67, 16)        0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 67, 67, 16)        64        
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 67, 67, 32)        2080      
_________________________________________________________________
max_pooling2d_2 (MaxPooling2 (None, 33, 33, 32)        0         
_________________________________________________________________
batch_normalization_2 (Batch (None, 33, 33, 32)        128       
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 33, 33, 64)        8256      
__________

In [4]:
# compile the model
rms_opt = keras.optimizers.RMSprop(lr=0.0001, rho=0.9, epsilon=None, decay=0.0)
piece_model.compile(loss='categorical_crossentropy', optimizer='rmsprop', 
                  metrics=['accuracy'])

STEP_SIZE_TRAIN = piece_train_iter.n/piece_train_iter.batch_size
STEP_SIZE_VALID = piece_valid_iter.n/piece_valid_iter.batch_size



In [5]:
from keras.callbacks import ModelCheckpoint   

# train the model
checkpointer = ModelCheckpoint(filepath='baseline_model.weights.best.hdf5', verbose=1, 
                               save_best_only=True)
piece_hist = piece_model.fit_generator(generator=piece_train_iter, 
                          steps_per_epoch=STEP_SIZE_TRAIN, 
                          validation_data=piece_valid_iter, 
                          validation_steps=STEP_SIZE_VALID,
                          epochs=100, 
                          callbacks=[checkpointer], 
                          verbose=2)

Epoch 1/100
Epoch 00000: val_loss improved from inf to 1.93214, saving model to baseline_model.weights.best.hdf5
3s - loss: 1.8489 - acc: 0.2541 - val_loss: 1.9321 - val_acc: 0.1843
Epoch 2/100
Epoch 00001: val_loss improved from 1.93214 to 1.92356, saving model to baseline_model.weights.best.hdf5
1s - loss: 1.7321 - acc: 0.3194 - val_loss: 1.9236 - val_acc: 0.1934
Epoch 3/100
Epoch 00002: val_loss improved from 1.92356 to 1.91037, saving model to baseline_model.weights.best.hdf5
1s - loss: 1.6642 - acc: 0.3409 - val_loss: 1.9104 - val_acc: 0.1480
Epoch 4/100
Epoch 00003: val_loss did not improve
1s - loss: 1.6332 - acc: 0.3470 - val_loss: 1.9207 - val_acc: 0.1994
Epoch 5/100
Epoch 00004: val_loss did not improve
1s - loss: 1.5918 - acc: 0.3691 - val_loss: 1.9608 - val_acc: 0.2326
Epoch 6/100
Epoch 00005: val_loss did not improve
1s - loss: 1.5738 - acc: 0.3674 - val_loss: 1.9731 - val_acc: 0.1782
Epoch 7/100
Epoch 00006: val_loss did not improve
1s - loss: 1.5472 - acc: 0.3960 - val_l

Epoch 00062: val_loss did not improve
1s - loss: 0.8529 - acc: 0.6861 - val_loss: 1.2661 - val_acc: 0.5468
Epoch 64/100
Epoch 00063: val_loss did not improve
1s - loss: 0.8183 - acc: 0.6912 - val_loss: 1.4817 - val_acc: 0.4894
Epoch 65/100
Epoch 00064: val_loss did not improve
1s - loss: 0.8506 - acc: 0.6903 - val_loss: 1.3025 - val_acc: 0.5438
Epoch 66/100
Epoch 00065: val_loss did not improve
1s - loss: 0.8140 - acc: 0.7008 - val_loss: 2.4885 - val_acc: 0.3625
Epoch 67/100
Epoch 00066: val_loss did not improve
1s - loss: 0.8226 - acc: 0.6944 - val_loss: 1.9648 - val_acc: 0.4562
Epoch 68/100
Epoch 00067: val_loss did not improve
1s - loss: 0.8024 - acc: 0.7134 - val_loss: 1.4092 - val_acc: 0.4894
Epoch 69/100
Epoch 00068: val_loss did not improve
1s - loss: 0.7943 - acc: 0.7158 - val_loss: 1.6772 - val_acc: 0.4773
Epoch 70/100
Epoch 00069: val_loss did not improve
1s - loss: 0.8021 - acc: 0.7082 - val_loss: 2.9725 - val_acc: 0.2931
Epoch 71/100
Epoch 00070: val_loss did not improve
1s

In [6]:
#Reset test iterator
STEP_SIZE_TEST = piece_test_iter.n/piece_test_iter.batch_size
piece_test_iter.reset()
# load the weights that yielded the best validation accuracy
piece_model.load_weights('baseline_model.weights.best.hdf5')
# evaluate and print test accuracy
score = piece_model.evaluate_generator(generator=piece_test_iter,steps=STEP_SIZE_TEST)
print('\n', 'Test accuracy:', score[1])


 Test accuracy: 0.594029853593


In [7]:
piece_test_iter.reset()
piece_pred = piece_model.predict_generator(piece_test_iter,steps=STEP_SIZE_TEST,verbose=1)

 9/10 [========================>.....] - ETA: 0s

In [8]:
predicted_class_indices=np.argmax(piece_pred,axis=1)
print(predicted_class_indices)

[2 0 2 2 0 2 5 0 2 5 0 0 0 0 0 0 0 2 0 0 0 0 1 0 0 0 2 0 0 0 0 0 0 0 0 0 5
 0 0 2 0 5 0 0 5 0 0 1 1 1 4 1 1 1 1 1 1 1 1 1 1 1 0 5 1 1 1 1 1 1 1 1 1 1
 1 1 5 1 1 5 2 0 5 5 2 5 2 0 2 0 5 2 5 0 5 2 2 0 0 2 0 2 1 2 2 2 1 0 0 0 2
 5 0 2 2 2 0 5 3 2 0 0 5 0 0 5 2 1 3 2 2 2 5 2 0 2 1 0 2 2 4 3 3 5 3 2 0 3
 3 5 0 3 3 0 3 5 3 5 3 2 0 3 5 2 3 3 5 5 2 5 3 5 5 3 5 5 3 0 3 0 3 3 5 3 3
 5 0 2 5 2 0 3 1 2 0 1 4 1 5 5 1 4 4 1 1 4 1 1 2 2 0 0 1 0 5 5 0 0 1 1 0 1
 2 4 5 5 2 5 2 5 5 5 2 2 5 3 2 1 5 2 5 5 2 2 3 5 2 5 0 0 5 5 5 3 5 1 2 2 5
 2 5 2 2 5 3 5 5 5 5 5 5 5 5 5 5 5 5 2 5 5 5 5 5 0 5 6 6 6 6 6 6 6 6 6 6 6
 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 3
 6 6]


In [9]:
labels = (piece_test_iter.class_indices)
labels = dict((v,k) for k,v in labels.items())
predictions = [labels[k] for k in predicted_class_indices]
truth =  [labels[k] for k in piece_test_iter.classes]

In [10]:
[labels[k] for k in piece_test_iter.classes]


['bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'bishop',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'king',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',
 'knight',

In [11]:
piece_test_iter.filenames

['bishop/1538778538.3017461.jpg',
 'bishop/1539025832.837544.jpg',
 'bishop/1539025486.6533895.jpg',
 'bishop/1538779519.5865016.jpg',
 'bishop/1538778643.4925923.jpg',
 'bishop/1539023363.830272.jpg',
 'bishop/1539025801.510389.jpg',
 'bishop/1538778410.6081142.jpg',
 'bishop/1538778215.8878355.jpg',
 'bishop/1538778752.467536.jpg',
 'bishop/1539025373.0113628.jpg',
 'bishop/1539023140.397658.jpg',
 'bishop/1539025334.2429533.jpg',
 'bishop/1538778214.3746974.jpg',
 'bishop/1539023732.561326.jpg',
 'bishop/1538777556.4695792.jpg',
 'bishop/1539024760.3120384.jpg',
 'bishop/1538778091.9695802.jpg',
 'bishop/1539024593.521976.jpg',
 'bishop/1538779155.2682924.jpg',
 'bishop/1538778833.610855.jpg',
 'bishop/1538777891.4931645.jpg',
 'bishop/1539017656.6888525_5.jpg',
 'bishop/1538778427.8668559.jpg',
 'bishop/1538777675.2653856.jpg',
 'bishop/1538778093.2397656.jpg',
 'bishop/1539024725.794323.jpg',
 'bishop/1539026035.275597.jpg',
 'bishop/1539024424.9014952.jpg',
 'bishop/1539024039.37

In [12]:
filenames=piece_test_iter.filenames
results=pd.DataFrame({"Filename":filenames,
                      "Truth": truth,
                      "Predictions":predictions})

In [13]:
results

,Filename,Predictions,Truth
0,bishop/1538778538.3017461.jpg,knight,bishop
1,bishop/1539025832.837544.jpg,bishop,bishop
2,bishop/1539025486.6533895.jpg,knight,bishop
3,bishop/1538779519.5865016.jpg,knight,bishop
4,bishop/1538778643.4925923.jpg,bishop,bishop
5,bishop/1539023363.830272.jpg,knight,bishop
6,bishop/1539025801.510389.jpg,rook,bishop
7,bishop/1538778410.6081142.jpg,bishop,bishop
8,bishop/1538778215.8878355.jpg,knight,bishop
9,bishop/1538778752.467536.jpg,rook,bishop
